<a href="https://colab.research.google.com/github/saumyasingh98/Lung-Cancer-Identification/blob/main/Lung_Cancer(VGG16).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install keras

In [ ]:
pip install tensorflow

     |████████████████████████████████| 462 kB 5.4 MB/s 


In [ ]:
from keras.layers import Input,Lambda,Dense,Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import os
from tensorflow.keras.optimizers import Adam, RMSprop
import matplotlib.pyplot as plt
from keras.utils.np_utils import to_categorical
from keras.layers import  Conv2D,MaxPooling2D,Activation,Dropout,Flatten,Dense,BatchNormalization
from keras.preprocessing.image import ImageDataGenerator,image,img_to_array,load_img
from keras.layers import AveragePooling2D

In [ ]:
train_dir = "/content/drive/MyDrive/lung _cancer_Dataset/train"
test_dir = "/content/drive/MyDrive/lung _cancer_Dataset/test"
val_dir = "/content/drive/MyDrive/lung _cancer_Dataset/valid"

In [ ]:
class_names = os.listdir(train_dir) # Get names of classes
class_name2id = { label: index for index, label in enumerate(class_names) } # Map class names to integer labels
class_name2id

{'bbps-0-1': 9,
 'bbps-2-3': 1,
 'cecum': 5,
 'dyed-lifted-polyps': 10,
 'dyed-resection-margins': 7,
 'esophagitis-a': 2,
 'polyps': 11,
 'pylorus': 4,
 'retroflex-rectum': 0,
 'retroflex-stomach': 6,
 'ulcerative-colitis-grade-2': 8,
 'z-line': 3}

In [ ]:
# Data Augmentation
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')
validation_generator = test_datagen.flow_from_directory(
        val_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

Found 6836 images belonging to 12 classes.
Found 930 images belonging to 12 classes.
Found 1951 images belonging to 12 classes.


In [ ]:
def build_model(backbone, lr=0.001):
    model = Sequential()
    model.add(backbone)
    #model.add(Flatten())
    model.add(Dense(4096, activation="relu"))
    #model.add(Dense(4096, activation="relu"))
    model.add(MaxPooling2D())
    model.add(Dropout(0.25))
    model.add(Dense(1000, activation="relu"))
    model.add(MaxPooling2D())
    model.add(Dense(128, activation="relu"))
    model.add(Flatten())
    #model.add(Dense(1,activation="sigmoid"))
    model.add(Dense(1,activation="sigmoid"))
    
    model.compile(
        loss='categorical_crossentropy',
        optimizer=Adam(lr=lr),
        metrics=['acc']
    )
    return model

vgg16 = VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(200, 200, 3)
)

model = build_model(vgg16 ,lr = 0.001)
model.summary()

58900480/58889256 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 6, 6, 512)         14714688  
                                                                 
 dense (Dense)               (None, 6, 6, 4096)        2101248   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 3, 3, 4096)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 3, 3, 4096)        0         
                                                                 
 dense_1 (Dense)             (None, 3, 3, 1000)        4097000   
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 1, 1, 1000)       0 

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
from sklearn.utils import class_weight
import numpy as np

class_weights = class_weight.compute_class_weight(
                                        class_weight = "balanced",
                                        classes = np.unique(train_generator.classes)
                ,
                                        y = train_generator.classes                                                    
                                    )
class_weights = dict(zip(np.unique(train_generator.classes), class_weights))
class_weights

{0: 1.2859292701279157,
 1: 0.6913430420711975,
 2: 0.7771714415643475,
 3: 0.7978524743230626,
 4: 0.8377450980392157,
 5: 2.1660329531051965,
 6: 0.7814357567443987,
 7: 0.8328460038986355,
 8: 2.1743002544529264,
 9: 1.0209080047789725,
 10: 1.8084656084656086,
 11: 0.9027997886951928}

In [ ]:
# Fit Model
history = model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=25,
    class_weight=class_weights,
    validation_data=test_generator,
    validation_steps=10
)

Epoch 1/25
100/100 [==============================] - 1785s 18s/step - loss: nan - accuracy: 0.0840 - val_loss: nan - val_accuracy: 0.0650
Epoch 2/25
100/100 [==============================] - 1750s 17s/step - loss: nan - accuracy: 0.0706 - val_loss: nan - val_accuracy: 0.0800
Epoch 3/25
100/100 [==============================] - 1754s 18s/step - loss: nan - accuracy: 0.0560 - val_loss: nan - val_accuracy: 0.0700
Epoch 4/25
100/100 [==============================] - 1761s 18s/step - loss: nan - accuracy: 0.0606 - val_loss: nan - val_accuracy: 0.0950
Epoch 5/25
100/100 [==============================] - 1754s 18s/step - loss: nan - accuracy: 0.0620 - val_loss: nan - val_accuracy: 0.0950
Epoch 6/25
100/100 [==============================] - 1755s 18s/step - loss: nan - accuracy: 0.0671 - val_loss: nan - val_accuracy: 0.0650
Epoch 7/25
 12/100 [==>...........................] - ETA: 24:54 - loss: nan - accuracy: 0.0667

In [ ]:
# model save
model.save_weights("vgg16-example-pneumonia.h5")

In [ ]:
# Visualize Loss and Accuracy Rates
fig, ax = plt.subplots(1, 2, figsize=(10, 3))
ax = ax.ravel()

for i, met in enumerate(['acc', 'loss']):
    ax[i].plot(history.history[met])
    ax[i].plot(history.history['val_' + met])
    ax[i].set_title('Model {}'.format(met))
    ax[i].set_xlabel('epochs')
    ax[i].set_ylabel(met)
    ax[i].legend(['train', 'val'])